# Topological Sort & Full Autograd

**Building LLMs from Scratch · Day 3** (following Andrej Karpathy's micrograd)

So far we've built a `Value` class that can compute gradients via the chain rule. But we had to call `_backward()` on each node **manually** in the correct order—one mistake and gradients are wrong.

Today we automate this: we'll use **topological sort** to determine the correct backward order automatically, then call `backward()` once on the loss and let the engine propagate gradients through the entire computation graph.

## 1. Introduction

**Automating backpropagation with topological sort**

The computation graph is a directed acyclic graph (DAG): each `Value` is a node, and edges go from inputs to outputs (e.g. `a` → `a*b`). To backpropagate correctly, we must call `_backward()` on a node only after we've called it on all nodes that depend on it (i.e. its children in the graph).

A **topological ordering** of a DAG visits nodes in an order such that for every edge `u → v`, `u` is visited before `v`. So in the *forward* direction: parents before children.

For **backward**, we need the reverse: parents after children. So we do a DFS post-order traversal (visit children first, then parent), collect nodes, then reverse the list. That gives us the correct order to call `_backward()`.

## 2. The Problem with Manual Backward

With manual backward, we must call `_backward()` in the right order by hand. One wrong order and gradients are wrong.

Example: `L = (a*b + c) * (a + b)`. The correct order depends on the graph structure—easy to get wrong.

In [ ]:
# Fragile manual approach: must call _backward in the right order
def manual_backward_example():
    """Simplified Value class for manual backward demo."""
    class Value:
        def __init__(self, data, _prev=()):
            self.data = data
            self.grad = 0.0
            self._prev = set(_prev)
            self._backward = lambda: None

    a, b, c = Value(2.0), Value(3.0), Value(1.0)
    ab = Value(a.data * b.data, (a, b))
    ab._backward = lambda: (
        setattr(a, 'grad', a.grad + b.data * ab.grad),
        setattr(b, 'grad', b.grad + a.data * ab.grad)
    )
    abc = Value(ab.data + c.data, (ab, c))
    abc._backward = lambda: (
        setattr(ab, 'grad', ab.grad + abc.grad),
        setattr(c, 'grad', c.grad + abc.grad)
    )
    apb = Value(a.data + b.data, (a, b))
    apb._backward = lambda: (
        setattr(a, 'grad', a.grad + apb.grad),
        setattr(b, 'grad', b.grad + apb.grad)
    )
    L = Value(abc.data * apb.data, (abc, apb))
    L._backward = lambda: (
        setattr(abc, 'grad', abc.grad + apb.data * L.grad),
        setattr(apb, 'grad', apb.grad + abc.data * L.grad)
    )

    L.grad = 1.0
    # Manual order: must call L before abc, apb; abc before ab, c; apb before a, b; ab before a, b
    L._backward()
    abc._backward()
    apb._backward()
    ab._backward()
    # c, a, b are leaves—no _backward

    print("Manual backward:", a.grad, b.grad, c.grad)

manual_backward_example()

## 3. Topological Sort via DFS

**Implement `build_topo` with DFS post-order traversal**

We traverse the graph in DFS order. For each node, we first recurse into its children (`_prev`), then append the node. This gives us a **post-order** traversal: children before parents.

Post-order yields [children..., parents...]. For backward, we must process a node only *after* all nodes that depend on it (its children in the graph). So we need children before parents—exactly what post-order gives. We iterate in **reversed post-order** (root first, then its dependencies) and call `_backward()` on each node; by the time we reach a node, all nodes that depend on it have already propagated their gradients.

In [ ]:
def build_topo(v, visited, topo):
    """
    DFS post-order traversal of the computation graph.
    Appends nodes to topo so that children appear before parents.
    Reversed topo = parents before children = correct order for backward.
    """
    if v in visited:
        return
    visited.add(v)
    for child in v._prev:
        build_topo(child, visited, topo)
    topo.append(v)


# Quick test: build a simple graph
class SimpleValue:
    def __init__(self, data, _prev=()):
        self.data = data
        self._prev = set(_prev)

a, b = SimpleValue(1.0), SimpleValue(2.0)
c = SimpleValue(a.data + b.data, (a, b))
d = SimpleValue(c.data * 2, (c,))

topo = []
build_topo(d, set(), topo)
print("Post-order (children before parents):", [id(x) for x in topo])
print("Reversed (for backward):", [id(x) for x in reversed(topo)])

## 4. The Complete Value Class

Full implementation with:
- `__init__` (data, grad, _backward, _prev, _op)
- `__add__` with _backward closure (grad += out.grad)
- `__mul__` with _backward closure (grad += other.data * out.grad)
- `__repr__`
- `backward()` method using topological sort
- Handle `isinstance` check for int/float operands

In [ ]:
class Value:
    """Scalar value with automatic differentiation via topological sort."""

    def __init__(self, data, _prev=(), _op=""):
        self.data = float(data)
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_prev)
        self._op = _op

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v in visited:
                return
            visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)

        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

## 5. Testing the Engine

Build expression `L = (a*b + c) * (a + b)`, call `L.backward()`, print all gradients.

In [ ]:
a = Value(2.0)
b = Value(3.0)
c = Value(1.0)

L = (a * b + c) * (a + b)
L.backward()

print("L =", L.data)
print("dL/da =", a.grad)
print("dL/db =", b.grad)
print("dL/dc =", c.grad)

## 6. Comparison: Manual vs Automated

Side by side: old manual 5-line backward vs new single `L.backward()`.

In [ ]:
print("=== Manual (fragile): ===")
a1, b1, c1 = Value(2.0), Value(3.0), Value(1.0)
L1 = (a1 * b1 + c1) * (a1 + b1)
L1.grad = 1.0
# Must call in exact order:
L1._backward()
for child in L1._prev:
    child._backward()
for child in L1._prev:
    for grandchild in child._prev:
        grandchild._backward()
# ... gets messy quickly for nested graphs
print("Manual dL/da:", a1.grad)

print("\n=== Automated (one line): ===")
a2, b2, c2 = Value(2.0), Value(3.0), Value(1.0)
L2 = (a2 * b2 + c2) * (a2 + b2)
L2.backward()
print("Automated dL/da:", a2.grad)
print("\nSame result:", abs(a1.grad - a2.grad) < 1e-9)

## 7. The Zero-Gradient Trap

Calling `backward()` twice accumulates gradients. Gradients are added with `+=`, so without zeroing, the second run adds to the first. Fix: zero grads before each backward pass.

In [ ]:
def zero_grad(v, visited=None):
    """Zero all gradients in the graph."""
    if visited is None:
        visited = set()
    if v in visited:
        return
    visited.add(v)
    v.grad = 0.0
    for child in v._prev:
        zero_grad(child, visited)


a, b, c = Value(2.0), Value(3.0), Value(1.0)
L = (a * b + c) * (a + b)

print("\n--- First backward ---")
L.backward()
print("a.grad:", a.grad)

print("\n--- Second backward (without zeroing) ---")
L.backward()  # Wrong! Gradients accumulate
print("a.grad (doubled!):", a.grad)

print("\n--- Second backward (with zero_grad first) ---")
zero_grad(L)
L.backward()
print("a.grad (correct):", a.grad)

## 8. Verification with PyTorch

Same expression with torch tensors, compare gradients.

In [ ]:
try:
    import torch
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False

if HAS_TORCH:
    a = torch.tensor(2.0, requires_grad=True)
    b = torch.tensor(3.0, requires_grad=True)
    c = torch.tensor(1.0, requires_grad=True)

    L = (a * b + c) * (a + b)
    L.backward()

    print("PyTorch:")
    print("  dL/da =", a.grad.item())
    print("  dL/db =", b.grad.item())
    print("  dL/dc =", c.grad.item())
    print("\nOur micrograd:")
    a2, b2, c2 = Value(2.0), Value(3.0), Value(1.0)
    L2 = (a2 * b2 + c2) * (a2 + b2)
    L2.backward()
    print("  dL/da =", a2.grad)
    print("  dL/db =", b2.grad)
    print("  dL/dc =", c2.grad)
    print("\nMatch:", torch.allclose(torch.tensor([a.grad, b.grad, c.grad]), torch.tensor([a2.grad, b2.grad, c2.grad])))
else:
    print("PyTorch not installed. Run: pip install torch")
    print("Our micrograd:")
    a2, b2, c2 = Value(2.0), Value(3.0), Value(1.0)
    L2 = (a2 * b2 + c2) * (a2 + b2)
    L2.backward()
    print("  dL/da =", a2.grad, "dL/db =", b2.grad, "dL/dc =", c2.grad)

---

**Building LLMs from Scratch** · [Day 3: Topological Sort & Autograd](https://omkarray.com/llm-day3.html)

← Prev: [Day 2](https://omkarray.com/llm-day2.html) · Next: [Day 4](https://omkarray.com/llm-day4.html) →